In [5]:
import numpy as np
from dataset.mnist import load_mnist
import matplotlib.pyplot as plt
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

t=np.array([0,0,1,0,0,0,0,0,0,0])
y=np.array([0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0])
print(mean_squared_error(y, t))

0.09750000000000003


In [6]:
def cross_entropy(y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

print(cross_entropy(y,t))

0.510825457099338


In [1]:
import sys,os
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True)

print(x_train.shape)
print(t_train.shape)

train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

(60000, 784)
(60000,)


In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t*np.log(y)) / batch_size

In [3]:
def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

def function_tmp1(x0):
    return x0**2 + 4.0**2

def function_tmp2(x1):
    return 3.**2 + x1 ** 2

print(numerical_diff(function_tmp2, 4.))

7.999999999999119


In [7]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)

    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)

        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val

    return grad

def function2(x):
    return x[0]**2 + x[1]**2


print(numerical_gradient(function2, np.array([3.0, 4.0])))
print(numerical_gradient(function2, np.array([0.0, 2.0])))
print(numerical_gradient(function2, np.array([3.0, 0.0])))

[6. 8.]
[0. 4.]
[6. 0.]


In [10]:
def gradient_decent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad

    return x

init_x = np.array([-3.0, 4.0])
print(gradient_decent(function2, init_x=init_x, lr=1e-10, step_num=100))

[-2.99999994  3.99999992]


In [ ]:
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

